产生与MINST数据集中与真实图片**风格**一致的图片

In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
 
# coding=utf-8
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
import os
 
# 创建文件夹
if not os.path.exists('./img'):
    os.mkdir('./img')
 
 
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)  # Clamp函数可以将随机变化的数值限制在一个给定的区间[min, max]内：
    out = out.view(-1, 1, 28, 28)  # view()函数作用是将一个多行的Tensor,拼接成一行，把向量变成矩阵的形式
    return out
 
 
batch_size = 128
num_epoch = 100
z_dimension = 100
# 图像预处理
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # (x-mean) / std
])
 
# mnist dataset mnist数据集下载
mnist = datasets.MNIST(
    root='./data/', train=True, transform=img_transform, download=True 
) #训练集
 
# data loader 数据载入
dataloader = torch.utils.data.DataLoader(
    dataset=mnist, batch_size=batch_size, shuffle=True
)
 
 
# 定义判别器  #####Discriminator######使用多层网络来作为判别器
# 将图片28x28展开成784，然后通过多层感知器，中间经过斜率设置为0.2的LeakyReLU激活函数，
# 最后接sigmoid激活函数得到一个0到1之间的概率进行二分类。
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(784, 256),  # 输入特征数为784，输出为256
            nn.LeakyReLU(0.2),  # 进行非线性映射
            nn.Linear(256, 256),  # 进行一个线性映射
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()  # 也是一个激活函数，二分类问题中，
            # sigmoid可以班实数映射到【0,1】，作为概率值，
            # 多分类用softmax函数
        )
 
    def forward(self, x):
        x = self.dis(x)
        return x
 
 
# ###### 定义生成器 Generator ##### x = G(z, θg)，输入一个随机的分布z，输出一个x（x属于Pg分布）
# 输入一个100维的0～1之间的高斯分布，然后通过第一层线性变换将其映射到256维,
# 然后通过ReLU激活函数，接着进行一个线性变换，再经过一个ReLU激活函数，
# 然后经过线性变换将其变成784维，最后经过Tanh激活函数是希望生成的假的图片数据分布
# 能够在-1～1之间。
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(100, 256),  # 用线性变换将输入映射到256维
            nn.ReLU(True),  # relu激活
            nn.Linear(256, 256),  # 线性变换
            nn.ReLU(True),  # relu激活
            nn.Linear(256, 784),  # 线性变换
            nn.Tanh()  # Tanh激活使得生成数据分布在【-1,1】之间，因为输入的真实数据的经过transforms之后也是这个分布
        )
 
    def forward(self, x):
        x = self.gen(x)
        return x
 
 
# 创建对象
D = discriminator()
G = generator()
if torch.cuda.is_available():
    D = D.cuda()
    G = G.cuda()
 
# 首先需要定义loss的度量方式  （二分类的交叉熵）
# 其次定义 优化函数,优化函数的学习率为0.0003
criterion = nn.BCELoss()  # 是单目标二分类交叉熵函数
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)
 
# ##########################进入训练##判别器的判断过程#####################
for epoch in range(num_epoch):  # 进行多个epoch的训练
    for i, (img, _) in enumerate(dataloader):
        num_img = img.size(0)
        # view()函数作用是将一个多行的Tensor,拼接成一行
        # 第一个参数是要拼接的tensor,第二个参数是-1
        # =============================训练判别器==================
        img = img.view(num_img, -1)  # 将图片展开为28*28=784
        real_img = Variable(img).cuda()  # 将tensor变成Variable放入计算图中
        real_label = Variable(torch.ones(num_img)).cuda()  # 定义真实的图片label为1
        fake_label = Variable(torch.zeros(num_img)).cuda()  # 定义假的图片的label为0
 
        # ########判别器训练train#####################
        # 分为两部分：1、真的图像判别为真；2、假的图像判别为假
        # 计算真实图片的损失
        real_out = D(real_img)  # 将真实图片放入判别器中
        real_out = real_out.squeeze()  # (128,1) -> (128,)
        d_loss_real = criterion(real_out, real_label)  # 得到真实图片的loss
        real_scores = real_out  # 得到真实图片的判别值，输出的值越接近1越好
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 随机生成一些噪声
        fake_img = G(z).detach()  # 随机噪声放入生成网络中，生成一张假的图片。 # 避免梯度传到G，因为G不用更新, detach分离
        fake_out = D(fake_img)  # 判别器判断假的图片，
        fake_out = fake_out.squeeze()  # (128,1) -> (128,)
        d_loss_fake = criterion(fake_out, fake_label)  # 得到假的图片的loss
        fake_scores = fake_out  # 得到假图片的判别值，对于判别器来说，假图片的损失越接近0越好
        # 损失函数和优化
        d_loss = d_loss_real + d_loss_fake  # 损失包括判真损失和判假损失
        d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
        d_loss.backward()  # 将误差反向传播
        d_optimizer.step()  # 更新参数
 
        # ==================训练生成器============================
        # ###############################生成网络的训练###############################
        # 原理：目的是希望生成的假的图片被判别器判断为真的图片，
        # 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
        # 反向传播更新的参数是生成网络里面的参数，
        # 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的
        # 这样就达到了对抗的目的
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 得到随机噪声
        fake_img = G(z)  # 随机噪声输入到生成器中，得到一副假的图片
        output = D(fake_img)  # 经过判别器得到的结果
        output = output.squeeze()
        g_loss = criterion(output, real_label)  # 得到的假的图片与真实的图片的label的loss
        # bp and optimize
        g_optimizer.zero_grad()  # 梯度归0
        g_loss.backward()  # 进行反向传播
        g_optimizer.step()  # .step()一般用在反向传播后面,用于更新生成网络的参数
 
        # 打印中间的损失
        if (i + 1) % 100 == 0:
            print('Epoch[{}/{}],d_loss:{:.6f},g_loss:{:.6f} '
                  'D real: {:.6f},D fake: {:.6f}'.format(
                epoch, num_epoch, d_loss.data.item(), g_loss.data.item(),
                real_scores.data.mean(), fake_scores.data.mean()  # 打印的是真实图片的损失均值
            )) #理论上当real_scores.data.mean()和fake_scores.data.mean()都为0.5时是训练效果最好的时候
        if epoch == 0 and i == len(dataloader)-1: # 记录一下真实的数据（最后一个batch）
            real_images = to_img(real_img.cpu().data)
            save_image(real_images, './img/real_images.png')
    if i == len(dataloader)-1:
        fake_images = to_img(fake_img.cpu().data)
        save_image(fake_images, './img/fake_images-{}.png'.format(epoch + 1))
 
# 保存模型
torch.save(G.state_dict(), './generator.pth')
torch.save(D.state_dict(), './discriminator.pth')

Epoch[0/100],d_loss:0.144313,g_loss:3.486530 D real: 0.967989,D fake: 0.100307
Epoch[0/100],d_loss:0.031358,g_loss:4.446863 D real: 0.997865,D fake: 0.028687
Epoch[0/100],d_loss:0.185937,g_loss:4.918364 D real: 0.962547,D fake: 0.127346
Epoch[0/100],d_loss:0.062624,g_loss:6.851132 D real: 0.988994,D fake: 0.045991
Epoch[1/100],d_loss:0.039476,g_loss:4.500678 D real: 0.994901,D fake: 0.033383
Epoch[1/100],d_loss:0.151536,g_loss:6.772314 D real: 0.955839,D fake: 0.071023
Epoch[1/100],d_loss:0.117181,g_loss:7.139060 D real: 0.968566,D fake: 0.066681
Epoch[1/100],d_loss:0.392790,g_loss:6.311606 D real: 0.842735,D fake: 0.058087
Epoch[2/100],d_loss:0.167021,g_loss:6.629264 D real: 0.937560,D fake: 0.052380
Epoch[2/100],d_loss:0.419147,g_loss:5.027452 D real: 0.920970,D fake: 0.143419
Epoch[2/100],d_loss:0.467001,g_loss:6.564289 D real: 0.829853,D fake: 0.053278
Epoch[2/100],d_loss:0.312081,g_loss:8.657451 D real: 0.883533,D fake: 0.045477
Epoch[3/100],d_loss:0.533541,g_loss:4.177750 D real:

Epoch[25/100],d_loss:0.403014,g_loss:3.332253 D real: 0.876668,D fake: 0.120749
Epoch[26/100],d_loss:0.366813,g_loss:3.813542 D real: 0.860976,D fake: 0.056349
Epoch[26/100],d_loss:0.461906,g_loss:2.997100 D real: 0.822825,D fake: 0.084134
Epoch[26/100],d_loss:0.274860,g_loss:3.522914 D real: 0.936404,D fake: 0.127159
Epoch[26/100],d_loss:0.305017,g_loss:4.137588 D real: 0.923163,D fake: 0.123772
Epoch[27/100],d_loss:0.229643,g_loss:2.798690 D real: 0.942511,D fake: 0.109280
Epoch[27/100],d_loss:0.402342,g_loss:4.013432 D real: 0.894545,D fake: 0.169040
Epoch[27/100],d_loss:0.630799,g_loss:4.158404 D real: 0.907590,D fake: 0.273873
Epoch[27/100],d_loss:0.319488,g_loss:3.685061 D real: 0.934034,D fake: 0.143561
Epoch[28/100],d_loss:0.363905,g_loss:4.429831 D real: 0.944067,D fake: 0.148555
Epoch[28/100],d_loss:0.505388,g_loss:3.296499 D real: 0.845062,D fake: 0.155388
Epoch[28/100],d_loss:0.396648,g_loss:2.858813 D real: 0.879402,D fake: 0.094392
Epoch[28/100],d_loss:0.309831,g_loss:4.5

Epoch[51/100],d_loss:0.576327,g_loss:2.507119 D real: 0.792220,D fake: 0.139435
Epoch[51/100],d_loss:0.578047,g_loss:1.989577 D real: 0.831629,D fake: 0.212883
Epoch[52/100],d_loss:0.631901,g_loss:2.355064 D real: 0.834017,D fake: 0.212209
Epoch[52/100],d_loss:0.800376,g_loss:3.180791 D real: 0.769228,D fake: 0.202079
Epoch[52/100],d_loss:0.476539,g_loss:3.085685 D real: 0.853949,D fake: 0.162837
Epoch[52/100],d_loss:0.634169,g_loss:1.870641 D real: 0.790430,D fake: 0.187205
Epoch[53/100],d_loss:0.496641,g_loss:2.804884 D real: 0.861311,D fake: 0.204600
Epoch[53/100],d_loss:0.569249,g_loss:3.091474 D real: 0.831089,D fake: 0.174522
Epoch[53/100],d_loss:0.678682,g_loss:2.744714 D real: 0.808703,D fake: 0.184675
Epoch[53/100],d_loss:0.657804,g_loss:2.639785 D real: 0.785750,D fake: 0.175325
Epoch[54/100],d_loss:0.538775,g_loss:3.043095 D real: 0.777312,D fake: 0.079287
Epoch[54/100],d_loss:0.611907,g_loss:2.818621 D real: 0.827947,D fake: 0.190734
Epoch[54/100],d_loss:0.385210,g_loss:3.1